In [1]:
from ultralytics import YOLO
from PIL import Image
import numpy as np
import os


model = YOLO("best.pt")

In [47]:
# calculate the IoU
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou


# used to read all the txt file in a folder
def read_txt_files(folder_path):
    # Get a list of all .txt file paths in the folder
    txt_file_paths = [
        os.path.join(folder_path, filename)
        for filename in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, filename)) and filename.lower().endswith('.txt')
    ]
    
    # Sort the list of file paths alphabetically by filename
    txt_file_paths.sort(key=lambda x: os.path.basename(x))
    
    content_list = []  # Initialize an empty list to store file contents
    
    # Read the contents of each file in sorted order
    for file_path in txt_file_paths:
        with open(file_path, 'r') as file:
            file_content = file.read()  # Read the entire content of the file
            if file_content == '':
                content_list.append(file_content)  # Append the content to the list
            else :
                string = file_content[3:-2]
                string_representation = string.replace(' ', ', ')
                content_list.append(string_representation)
    return content_list

def read_img_files_size(folder_path):
    # Get a list of all .PNG file paths in the folder
    img_file_paths = [
        os.path.join(folder_path, filename)
        for filename in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, filename)) and filename.lower().endswith('.png')
    ]
    
    # Sort the list of file paths alphabetically by filename
    img_file_paths.sort(key=lambda x: os.path.basename(x))
    
    # List to store tuples of (file_path, image_size)
    image_data_list = []
    
    for file_path in img_file_paths:
        try:
            # Open the image file using PIL
            with Image.open(file_path) as img:
                # Get the dimensions (width, height) of the image
                image_size = img.size  # This will return a tuple (width, height)
                # Append (image_size) tuple to the list
                image_data_list.append(image_size)
        except Exception as e:
            print(f"Error processing image {file_path}: {e}")
    
    return image_data_list

# change yolo (x,y,w,h) bbox to bbox (x1, y1, x2, y2)
def yolobbox2bbox(x,y,w,h,img_w,img_h):
    w = w * img_w
    h = h * img_h
    x = x * img_w
    y = y * img_h
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    x1 = round(x1, 5)
    y1 = round(y1, 5)
    x2 = round(x2, 5)
    y2 = round(y2, 5)
    return [x1, y1, x2, y2]

# the list from yolo predict and ground truth are a string list
# change it to float
def string_to_float_list(list_string):
    # Check if the input string is empty
    if not list_string.strip():  # strip() removes leading/trailing whitespace
        return []  # Return an empty list if the string is empty
    # Step 1: Split the string by commas to separate values
    values_str = list_string.split(',')
    # Step 2: Convert each split string to a float
    values_float = [float(val.strip()) for val in values_str]
    # Step 3: Return the list of float values
    return values_float


# results can be obtain from yolo predict
def list_of_predicted(results):
    labels = []
    for result in results: 
        boxes = result.boxes  # Boxes object for bounding box outputs 

        # Convert each element in the array to a formatted string
        formatted_strings = [f"{value:.4f}" for value in boxes.xyxy.flatten()]

        # Join the formatted strings into a single string separated by commas
        string_representation = ", ".join(formatted_strings)
        labels.append(string_representation)
        
    return labels

In [48]:
# Usage
ground_truth = read_txt_files('dataset/labels/test')
imgsize = read_img_files_size('dataset/images/test')

labels = list_of_predicted(results)

In [49]:
i = 0
for x in ground_truth:
    if x == '':
        i = i + 1
print(i)

467


In [53]:
size = len(labels)
count = 0
for i in range (size):
    if (len(labels[i]) != 0) and (len(ground_truth[i]) != 0):
        label = string_to_float_list(labels[i])
        temp = string_to_float_list(ground_truth[i])
        if len(temp) != 0:
            res = yolobbox2bbox(temp[0], temp[1], temp[2], temp[3], imgsize[i][0], imgsize[i][1])
        else:
            res = temp
        if bb_intersection_over_union(res, label) > 0.9:
            count = count + 1
    elif (len(labels[i]) == 0) and (len(ground_truth[i]) == 0):
        count = count + 1

In [51]:
count/size

0.38638297872340427

In [52]:
count

454

In [54]:
frame = 469

gt = string_to_float_list(labels[frame])
temp = string_to_float_list(ground_truth[frame])
if len(temp) != 0:
    res = yolobbox2bbox(temp[0], temp[1], temp[2], temp[3], imgsize[frame][0], imgsize[frame][1])
else:
    res = temp
print(gt)
print(res)

[1135.2764, 389.2028, 1166.3369, 420.4973]
[1133.24928, 387.02988, 1168.91904, 421.44948]


In [55]:
bb_intersection_over_union(res, gt)

0.7917178807118277

In [3]:
results = model.predict(source="dataset/images/test", save=True, classes=32) #classes=32



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000000.PNG: 480x640 (no detections), 4097.7ms
image 2/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000001.PNG: 480x640 (no detections), 3665.1ms
image 3/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000002.PNG: 480x640 (no detections), 3438.3ms
image 4/1175 /Users/arvinn/Documents

image 56/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000055.PNG: 480x640 (no detections), 2887.7ms
image 57/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000056.PNG: 480x640 (no detections), 2905.0ms
image 58/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000057.PNG: 480x640 (no detections), 2896.8ms
image 59/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000058.PNG: 480x640 (no detections), 3007.4ms
image 60/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000059.PNG: 480x640 (no detections), 2859.1ms
image 61/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000060.PNG: 480x640 (no detections), 2907.9ms
image 62/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000061.PNG: 480x640 (no detections), 2839.7ms
image 63/1175

image 115/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000114.PNG: 480x640 (no detections), 2917.9ms
image 116/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000115.PNG: 480x640 (no detections), 3013.7ms
image 117/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000116.PNG: 480x640 (no detections), 2840.5ms
image 118/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000117.PNG: 480x640 (no detections), 4413.1ms
image 119/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000118.PNG: 480x640 (no detections), 5268.7ms
image 120/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000119.PNG: 480x640 (no detections), 5256.4ms
image 121/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000120.PNG: 480x640 (no detections), 4871.0ms
image 

image 173/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000172.PNG: 480x640 1 sports ball, 3121.7ms
image 174/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000173.PNG: 480x640 1 sports ball, 3214.0ms
image 175/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000174.PNG: 480x640 1 sports ball, 3888.1ms
image 176/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000175.PNG: 480x640 1 sports ball, 3406.7ms
image 177/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000176.PNG: 480x640 (no detections), 3241.6ms
image 178/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000177.PNG: 480x640 1 sports ball, 2899.9ms
image 179/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000178.PNG: 480x640 1 sports ball, 3024.9ms
image 180/1175 /Us

image 232/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000231.PNG: 480x640 (no detections), 3064.0ms
image 233/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000232.PNG: 480x640 (no detections), 6835.8ms
image 234/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000233.PNG: 480x640 (no detections), 5976.8ms
image 235/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000234.PNG: 480x640 (no detections), 4274.1ms
image 236/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000235.PNG: 480x640 (no detections), 2923.8ms
image 237/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000236.PNG: 480x640 (no detections), 2873.3ms
image 238/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000237.PNG: 480x640 (no detections), 2866.4ms
image 

...

image 886/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000885.PNG: 384x640 1 sports ball, 2703.8ms
image 887/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000886.PNG: 384x640 1 sports ball, 2720.8ms
image 888/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000887.PNG: 384x640 (no detections), 2648.2ms
image 889/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000888.PNG: 384x640 (no detections), 2673.8ms
image 890/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000889.PNG: 384x640 1 sports ball, 2677.8ms
image 891/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000890.PNG: 384x640 1 sports ball, 3316.0ms
image 892/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000891.PNG: 384x640 1 sports ball, 3637.8ms
image 893/1175 /

image 944/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000943.PNG: 384x640 1 sports ball, 2841.4ms
image 945/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000944.PNG: 384x640 1 sports ball, 2819.2ms
image 946/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000945.PNG: 384x640 1 sports ball, 2845.8ms
image 947/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000946.PNG: 384x640 (no detections), 2828.1ms
image 948/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000947.PNG: 384x640 1 sports ball, 3355.7ms
image 949/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000948.PNG: 384x640 1 sports ball, 3297.3ms
image 950/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_000949.PNG: 384x640 1 sports ball, 3261.6ms
image 951/1175 /Us

image 1003/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001002.PNG: 384x640 (no detections), 2662.6ms
image 1004/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001003.PNG: 384x640 (no detections), 2648.9ms
image 1005/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001004.PNG: 384x640 (no detections), 2676.5ms
image 1006/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001005.PNG: 384x640 (no detections), 2811.0ms
image 1007/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001006.PNG: 384x640 (no detections), 2693.7ms
image 1008/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001007.PNG: 384x640 (no detections), 3174.5ms
image 1009/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001008.PNG: 384x640 1 sports ball, 2759.8ms
i

image 1061/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001060.PNG: 384x640 1 sports ball, 2691.6ms
image 1062/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001061.PNG: 384x640 1 sports ball, 2717.5ms
image 1063/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001062.PNG: 384x640 1 sports ball, 2862.4ms
image 1064/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001063.PNG: 384x640 1 sports ball, 3748.0ms
image 1065/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001064.PNG: 384x640 1 sports ball, 3946.0ms
image 1066/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001065.PNG: 384x640 1 sports ball, 2778.6ms
image 1067/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001066.PNG: 384x640 1 sports ball, 3010.2ms
image 1068/11

image 1120/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001119.PNG: 384x640 (no detections), 2677.5ms
image 1121/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001120.PNG: 384x640 1 sports ball, 2679.4ms
image 1122/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001121.PNG: 384x640 1 sports ball, 2699.0ms
image 1123/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001122.PNG: 384x640 1 sports ball, 2708.5ms
image 1124/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001123.PNG: 384x640 1 sports ball, 2736.0ms
image 1125/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001124.PNG: 384x640 1 sports ball, 2929.1ms
image 1126/1175 /Users/arvinn/Documents/Arvin UW/WINTER2024/EE497/YOLO/dataset/images/test/frame_001125.PNG: 384x640 1 sports ball, 2902.5ms
image 1127/